<a href="https://colab.research.google.com/github/raina-11/streamlit-webapp/blob/main/NST_streamlit_webapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit

     |████████████████████████████████| 7.4MB 2.7MB/s 
     |████████████████████████████████| 6.7MB 35.9MB/s 
     |████████████████████████████████| 102kB 11.1MB/s 
     |████████████████████████████████| 112kB 34.8MB/s 
     |████████████████████████████████| 133kB 39.1MB/s 
     |████████████████████████████████| 163kB 41.5MB/s 
     |████████████████████████████████| 4.4MB 35.7MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 71kB 3.0MB/s 
     |████████████████████████████████| 122kB 43.8MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=ad7ff0e3f4f4b1e84db87ac76d1c9b43cdd0fcbf7f472034528453a6c5493a9c
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=2693e40940cb4b59968d1a6a21f43700adc52a9034825a19f5f299bd3a575031
  Stored in directory: /root/.c

In [2]:
!pip install pyngrok

  Created wheel for pyngrok: filename=pyngrok-4.2.2-cp36-none-any.whl size=18025 sha256=3f40eee14ee99edc8f27458f742ab82302740f337c51c27d4d5c17e7497a1dcb
  Stored in directory: /root/.cache/pip/wheels/9c/d3/b5/502fbef4b2782a6142bc00c62ccde76ee04c9148fb43b1b2cd
Successfully built pyngrok


In [14]:
%%writefile app.py

import streamlit as st
from PIL import Image
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def modelf():
  modelx=models.vgg19(pretrained=True).features
  return modelx

def freeze():
  #freezing the parameters in vgg
  model=modelf()
  for param in model.parameters():
    param.requires_grad_(False)

def img_transformf(content_img,style_img):
  img_transform=transforms.Compose([transforms.Resize((400,400)),transforms.ToTensor(),transforms.Normalize((0.485,0.456, 0.406),(0.229,0.224, 0.225))])
  content=img_transform(content_img)[:3,:,:].unsqueeze(0).to(device)
  style=img_transform(style_img).unsqueeze(0).to(device)
  return content,style



def getfeatures(image,model,layers=None):
  if layers is None:
    layers = {'0': 'conv1_1',
                    '5': 'conv2_1', 
                    '10': 'conv3_1', 
                    '19': 'conv4_1',
                    '21': 'conv4_2', 
                    '28': 'conv5_1'}
  features={}
  x=image
  for name,layer in model._modules.items():
    x=layer(x)
    if name in layers:
      features[layers[name]]=x
  return features

def Gram_Matrix(tensor):
  n,d,h,w=tensor.size()
  tensor=tensor.view(d,h*w)
  grammat=torch.mm(tensor,tensor.t())
  return grammat



def im_convert(tensor):


  image=tensor.to("cpu").clone().detach()
  image = image.numpy().squeeze()
  image=image.transpose(1,2,0)
  image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
  image=image.clip(0,1)

  return image




def main():

  st.title("NST Classifier")
  st.header("CHOOSE CONTENT IMAGE")
  uploaded_file=st.file_uploader("Choose the content image",type=["jpg","png","jpeg"])
  if uploaded_file is None:
    st.text("choose file")
    #imarray = np.random.rand(100,100,3) * 255
    #content_imgi = Image.fromarray(imarray.astype('uint8')).convert('RGBA') 
  else:
    content_imgi=Image.open(uploaded_file).convert('RGB')
    st.image(content_imgi,width=400,caption="Content Image")
    img_transform=transforms.Compose([transforms.Resize((400,400)),transforms.ToTensor(),transforms.Normalize((0.485,0.456, 0.406),(0.229,0.224, 0.225))])
    content=img_transform(content_imgi).unsqueeze(0).to(device)
   # content_imgi = (content_imgi.astype(np.float32) / 255.0)
   # content_imgi = content_imgi[np.newaxis,...]

  st.header("CHOOSE STYLE IMAGE")
  ufile=st.file_uploader("Choose the style image",type=["jpg","png","jpeg"])
  if ufile is None:
    st.text("choose file")
    #imarray = np.random.rand(100,100,3) * 255
    #style_imgi = Image.fromarray(imarray.astype('uint8')).convert('RGBA') 
  else:
    style_imgi=Image.open(ufile)
    st.image(style_imgi,width=400,caption="Style Image")
    img_transform=transforms.Compose([transforms.Resize((400,400)),transforms.ToTensor(),transforms.Normalize((0.485,0.456, 0.406),(0.229,0.224, 0.225))])
    style=img_transform(style_imgi).unsqueeze(0).to(device)
  if st.button("click to produce Stylised image"):

    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model=modelf()
    model.to(device)
    freeze()
    #content,style=img_transformf(content_imgi,style_imgi)
    feature_layers = {'0': 'conv1_1',
            '5': 'conv2_1', 
            '10': 'conv3_1', 
            '19': 'conv4_1',
            '21': 'conv4_2', 
            '28': 'conv5_1'}
    content_features={}
    img=content
    for name,layer in model._modules.items():
      img=layer(img)
      if name in feature_layers:
        content_features[feature_layers[name]]=img
    style_features={}
    img=style
    for name,layer in model._modules.items():
      img=layer(img)
      if name in feature_layers:
        style_features[feature_layers[name]]=img

    style_gram_matrix={layer: Gram_Matrix(style_features[layer]) for layer in style_features}
    target=content.clone().requires_grad_(True).to(device)
    style_weights = {'conv1_1': 1.,
                      'conv2_1': 0.75,
                      'conv3_1': 0.2,
                      'conv4_1': 0.2,
                      'conv5_1': 0.2}

    content_weight = 1  # this is alpha in the final loss
    style_weight = 1e4  # this is beta in the final loss

    optimizer = optim.Adam([target], lr=0.003)
    steps = 1500 
    show=500 

    for ii in range(1, steps+1):
      target_features = getfeatures(target, model)
      content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2'])**2)
      style_loss = 0
      for layer in style_weights:
        target_feature = target_features[layer]
        target_gram = Gram_Matrix(target_feature)
        _, d, h, w = target_feature.shape
        style_gram = style_gram_matrix[layer]
        layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram)**2)
        style_loss += layer_style_loss / (d * h * w)
              
      total_loss = content_weight * content_loss + style_weight * style_loss
      optimizer.zero_grad()
      total_loss.backward(retain_graph=True)
      optimizer.step()
  
    st.success("Stylised Image:")   
    st.image(im_convert(target), caption="final image")
    #st.success("Run successfuly")


if __name__ == '__main__':
	main()



Overwriting app.py


In [4]:
!ls


app.py	sample_data


In [5]:
!ngrok authtoken 1ism5QubiIvBxrmFaVHNUeSxJFF_p4a1rhHYMYSobKksuJb9

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
!streamlit run app.py &>/dev/null&

In [7]:
!ngrok

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [8]:
from pyngrok import ngrok

In [9]:
!pgrep streamlit

193


In [10]:
public_url = ngrok.connect(port='8501')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Support for "return_ngrok_tunnel" as "False" is deprecated and will be removed in 5.0.0, when this method will return a NgrokTunnel instead of a str
  """Entry point for launching an IPython kernel.


In [11]:
public_url

'http://ab2977fff1b4.ngrok.io'